In [1]:
from datasets import load_dataset

# load from parquet file (~4000 samples in a parquet file)
# link to other parquet files: https://huggingface.co/datasets/linhtran92/viet_bud500/tree/main/data

train_url = "https://huggingface.co/datasets/linhtran92/viet_bud500/resolve/main/data/train-00000-of-00105-be5f872f8be772f5.parquet"
test_url = "https://huggingface.co/datasets/linhtran92/viet_bud500/resolve/main/data/test-00000-of-00002-531c1d81edb57297.parquet"

data_files = {"train": train_url, "test" : test_url}
dataset = load_dataset("parquet", data_files=data_files, )

# # load dataset via streaming
# dataset = load_dataset("linhtran92/viet_bud500", split='test', streaming=True)
# dataset.take(2)

# # load all (649158 samples, ~100gb, ~2hrs to complete loading)
# dataset = load_dataset("linhtran92/viet_bud500", split="test")

In [2]:
import soundfile as sf

In [22]:
# def convert_waveform_to_audio(waveform_list, sampling_rate=24000, path_to_audio='',prefix='train'):
#     count = 0
#     for waveform in waveform_list:
#         print(waveform)
#         sf.write(f'{path_to_audio}{prefix}_{str(count).zfill(10)}.wav', waveform['array'], sampling_rate)
#         count += 1
#     print('saved all audio files!') 
# sampling_rate = 24000
# for audio in dataset['train']['audio']:
#     sf.write(f'../dataset/audio/train_24k/train24k_{str(count).zfill(10)}.wav', audio['array'], sampling_rate)
#     count += 1
# count = 0
# for audio in dataset['test']['audio']:
#     sf.write(f'../dataset/audio/test_24k/test24k_{str(count).zfill(10)}.wav', audio['array'], sampling_rate)
#     count += 1

In [3]:
import scipy.signal
import soundfile as sf

def upsample_waveform(waveform, orig_sampling_rate, target_sampling_rate):
    # Tính tỷ lệ upsampling
    resample_factor = target_sampling_rate / orig_sampling_rate
    
    # Upsample tín hiệu bằng cách nội suy
    upsampled_waveform = scipy.signal.resample(waveform, int(len(waveform) * resample_factor))
    
    return upsampled_waveform

def convert_waveform_to_audio(waveform_list, orig_sampling_rate, target_sampling_rate=24000, path_to_audio='', prefix='train'):
    count = 0
    for waveform in waveform_list:
        # Upsample lên 24kHz
        upsampled_waveform = upsample_waveform(waveform['array'], orig_sampling_rate, target_sampling_rate)
        
        # Lưu tệp âm thanh với sampling rate 24kHz
        sf.write(f'{path_to_audio}{prefix}_{str(count).zfill(10)}.wav', upsampled_waveform, target_sampling_rate)
        count += 1
    print('saved all audio files!')


In [4]:
%ls

 Volume in drive D has no label.
 Volume Serial Number is 6F86-43F0

 Directory of d:\AI_CODE\BunStyleTTS2\notebook

09/11/2024  07:47 PM    <DIR>          .
09/05/2024  02:40 PM    <DIR>          ..
09/12/2024  05:59 PM           642,128 preprocessing.ipynb
               1 File(s)        642,128 bytes
               2 Dir(s)  81,914,552,320 bytes free


In [7]:
%cd ../

d:\AI_CODE\BunStyleTTS2


In [8]:
# convert_waveform_to_audio(dataset['train']['audio'], sampling_rate=24000,path_to_audio='../dataset/audio/train_24k/',prefix='train24k')
convert_waveform_to_audio(dataset['test']['audio'], orig_sampling_rate=16000, target_sampling_rate=24000,path_to_audio='dataset/audio/test_24k_re/',prefix='test24k_re')

saved all audio files!


In [4]:
import phonemizer
global_phonemizer = phonemizer.backend.EspeakBackend(language='vi', preserve_punctuation=True,  with_stress=True)

In [6]:
from phonemizer import phonemize
text = "Xin chào, tôi là một lập trình viên."

# Sử dụng phonemizer để chuyển đổi văn bản thành IPA
ipa_text = phonemize(text, language='vi', backend='espeak', strip=True, preserve_punctuation=True)

print(ipa_text)

sin tʃaː2w, t̪oj laː2 mo6t̪ lə6p tʃi2ɲ viɛn.


In [30]:
def create_data_list(transcription_list, path_to_audio, path_to_list, prefix, language, max):
    count = 0
    global_phonemizer = phonemizer.backend.EspeakBackend(language=language, preserve_punctuation=True,  with_stress=True)
    with open(path_to_list, 'w') as f:
        for text in transcription_list:
            text = text.strip()
            ps = global_phonemizer.phonemize([text])
            f.write(f'{path_to_audio}/{prefix}_{str(count).zfill(10)}.wav|{ps[0]}|0\n')
            count += 1
            if (count == max):
                break

# with open('../dataset/train_list_24k.txt', 'w') as f:
#     count = 0
#     for text in dataset['train']['transcription']:
#         text = text.strip()
#         ps = global_phonemizer.phonemize([text])
#         f.write(f'train_24k/train24k_{str(count).zfill(10)}.wav|{ps[0]}|0\n')
#         count += 1
# with open('../dataset/test_list_24k.txt', 'w') as f:
#     count = 0
#     for text in dataset['test']['transcription']:
#         text = text.strip()
#         ps = global_phonemizer.phonemize([text])
#         f.write(f'test_24k/test24k_{str(count).zfill(10)}.wav|{ps[0]}|0\n')
#         count += 1

In [34]:
create_data_list(dataset['train']['transcription'],'train_24k','dataset/train_list_24k_small.txt','train24k','vi',100)

In [35]:
create_data_list(dataset['test']['transcription'],'test_24k','dataset/test_list_24k_small.txt','test24k','vi',100)